In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [3]:
# 读取8月份的数据
cvjd_202208_6108 = pd.read_pickle('../generalization_data/cvjd_202208_6108.pkl').reset_index(drop=True)
cvjd_202208_6108

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,8a69c41582926ae60182a07b24c1048c,8a69f62f82926a8601829f1379a07648,2,2,4.0,2022-08
1,8a69c41582926ae60182a0e09bf27664,8a69d77e82926b410182a083fbf602c0,3,3,4.0,2022-08
2,8a69c41582926ae60182a119de713808,8a69f68782590ecb01826326b5e94919,-1,2,4.0,2022-08
3,8a69c41582a3cc240182b0354c0e7bad,2c92071581ebf3390181ec9e500755e4,1,1,4.0,2022-08
4,8a69c4a3827cc886018284b9612d0f03,8a69f601823a92b0018243dcb9c45f7a,-1,1,4.0,2022-08
...,...,...,...,...,...,...
6103,8a69d71b82add4e10182aebff7d016d2,8a69c468826ee9b4018290a64a0c01bb,2,2,4.0,2022-08
6104,8a69d7b4826d7dfb01826d7efad200f1,2c920715818a462a01818e3f9c0a2337,3,3,4.0,2022-08
6105,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,-1,1,5.0,2022-08
6106,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,3,3,4.0,2022-08
